In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
%matplotlib inline

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam

In [ ]:
########## prepare data #########################
# load MNIST and split between training  and testing data sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('training data:', x_train.shape[0], 'images with', x_train.shape[1], 'times', x_train.shape[2], 'pixels')
print('test data:', x_test.shape[0], 'images with', x_test.shape[1], 'times', x_test.shape[2], 'pixels \n')

In [ ]:
# convert images into 1D vectors with 28^2 = 784 components 
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# normalize features to float in range 0..1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('training data:', x_train.shape[0], 'images with', x_train.shape[1], 'features')
print('test data:', x_test.shape[0], 'images with', x_test.shape[1], 'features')

In [ ]:
# convert target values to one hot vectors 
print('original y data example:', y_train[0])
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
print('after converting to a one hot vector:', y_train[0])

In [ ]:
# preview the images
plt.figure(figsize=(12,10))
x, y =5, 4
for i in range(20):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_train[i].reshape((28,28)),cmap='gray')
    plt.title('target: {}'.format(np.argmax(y_train[i])))
    plt.axis('off')
plt.show()

# let's start with a simple model

In [ ]:
# design a simple neural network
model = Sequential()
model.add(Dense(15, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer= Adam(),                             
              metrics=['accuracy'])

#### First layer has 15 Neurons which are each connected to 784 pixels -> this corresponds to 11760 weight. Additionally we need 15 biases for the 15 neurons.

#### Second layer has 10 (output) neurons, each connected to 15 neurons from the first layer -> 150 weights plus 10 biases

In [ ]:
# train the model
batch_size = 128
epochs = 20

#history simple will save our training results
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,              
                    validation_data=(x_test, y_test))

In [ ]:
# evaluate model performance
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: {:.4f}".format(score[0]))
print("Test accuracy: {:.4f}".format(score[1]))

In [ ]:
# show the learning process
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))


plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'bo', label='Training')
plt.plot(epochs, val_accuracy, 'r', label='Test')
plt.xlabel('Epoch', size=18)
plt.ylabel('Accuracy', size=18)
plt.xticks(np.arange(0, 21, step=5))
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(epochs, loss, 'bo', label='Training')
plt.plot(epochs, val_loss, 'r', label='Test')
plt.xlabel('Epoch', size=18)
plt.ylabel('Loss', size=18)
plt.xticks(np.arange(0, 21, step=5))
plt.legend()
plt.show()

In [ ]:
# let's have a look at the results
# make predictions for all test data
y_pred = model.predict(x_test)

In [ ]:
# pick an example from the test data
n = 0

np.set_printoptions(precision=1)
print('predicted probabilities: ', y_pred[n])



plt.figure(figsize=(5,5))
plt.imshow(x_test[n].reshape((28,28)),cmap='gray')
plt.title('target: {} predicted: {}'.format(np.argmax(y_test[n]), np.argmax(y_test[n]), size=16))
plt.axis('off')
plt.show()

In [ ]:
# convert one hot back to vector
print(y_test.shape)
Y_test = np.argmax(y_test, axis = 1)
Y_pred = np.argmax(y_pred, axis = 1)
print(Y_test.shape)

In [ ]:
# show the result
plt.figure(figsize=(12,12))
x, y =5, 4
for i in range(20):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_test[i].reshape((28,28)),cmap='gray')
    plt.title('target: {}\npredicted: {}'.format(Y_test[i], Y_pred[i]))
    plt.axis('off')
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
from sklearn import metrics
cm1 = metrics.confusion_matrix(Y_test, Y_pred)

plt.figure(figsize=(9,9))
sns.heatmap(cm1, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues_r')
plt.ylabel('true number', size=17)
plt.xlabel('predicted number', size=17)
plt.show()

In [ ]:
# let's have a look at the errors
errors = (Y_pred - Y_test != 0)
error_pos = np.array(np.where(errors))
print('total number of errors:', len(error_pos[0,:]))

In [ ]:
# show the errors
plt.figure(figsize=(12,12))
x, y =5, 4
for i in range(20):  
    plt.subplot(y, x, i+1)
    idx = error_pos[0,i] 
    plt.imshow(x_test[idx].reshape((28,28)),cmap='gray')
    plt.title('target: {}\npredicted: {}'.format(Y_test[idx], Y_pred[idx]))
    plt.axis('off')
plt.show()

### in class exercise: 
create a second model named model_2 with 32 neurons in a first hidden layer, 16 neurons in a second hidden layer, and again 10 output neurons. Train the new model for 20 epochs (don't forget to compile it) and plot the learning curve.

In [ ]:
# show the learning process
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))


plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'bo', label='Training')
plt.plot(epochs, val_accuracy, 'r', label='Test')
plt.xlabel('Epoch', size=18)
plt.ylabel('Accuracy', size=18)
plt.xticks(np.arange(0, 21, step=5))
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(epochs, loss, 'bo', label='Training')
plt.plot(epochs, val_loss, 'r', label='Test')
plt.xlabel('Epoch', size=18)
plt.ylabel('Loss', size=18)
plt.xticks(np.arange(0, 21, step=5))
plt.legend()
plt.show()

In [ ]:
# evaluate model performance
score = model_2.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: {:.4f}".format(score[1]))